In [1]:
import numpy as np
import tensorflow as tf
from tensorflow.keras import models, layers
import matplotlib.pyplot as plt

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
BATCH_SIZE = 32
IMAGE_SIZE = 128
train_dataset = tf.keras.preprocessing.image_dataset_from_directory(
    "/content/drive/MyDrive/data set/train",
    shuffle = True,
    image_size = (IMAGE_SIZE,IMAGE_SIZE),
    batch_size = BATCH_SIZE 
)

In [ ]:
test_dataset = tf.keras.preprocessing.image_dataset_from_directory(
    "/content/drive/MyDrive/data set/test",
    shuffle = True,
    image_size = (IMAGE_SIZE,IMAGE_SIZE),
    batch_size = BATCH_SIZE  
)

In [5]:
class_names = train_dataset.class_names
num_class=len(class_names)

In [ ]:
class_names

In [7]:
def get_dataset_partitions_tf(ds, train_split = 0.8,val_split = 0.2,shuffle = True,shuffle_size = 10000):
  assert(train_split+val_split) == 1
  ds_size = len(ds)
  if shuffle:
    ds = ds.shuffle(shuffle_size, seed = 12)
  train_size = int(train_split*ds_size)
  val_size = int(val_split * ds_size)

  train_ds = ds.take(train_size)
  val_ds = ds.skip(train_size).take(val_size)
  return train_ds, val_ds

In [8]:
train_ds, val_ds = get_dataset_partitions_tf(train_dataset)

In [ ]:
train_ds

In [10]:
train_ds = train_ds.cache().shuffle(1000).prefetch(buffer_size=tf.data.AUTOTUNE)
val_ds_ds = val_ds.cache().shuffle(1000).prefetch(buffer_size=tf.data.AUTOTUNE)
test_ds = test_dataset.cache().shuffle(1000).prefetch(buffer_size=tf.data.AUTOTUNE)

In [11]:
resize_and_rescale = tf.keras.Sequential([
                     layers.experimental.preprocessing.Resizing(IMAGE_SIZE,IMAGE_SIZE),
                     layers.experimental.preprocessing.Rescaling(1.0/255)
])

In [12]:
data_augmentation = tf.keras.Sequential([
                     layers.experimental.preprocessing.RandomFlip("horizontal_and_vertical"),
                     layers.experimental.preprocessing.RandomRotation(0.2),
])

In [13]:
train_ds = train_ds.map(
    lambda x, y: (data_augmentation(x, training=True), y)
).prefetch(buffer_size=tf.data.AUTOTUNE)

In [14]:
from keras.models import Sequential
from keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout, BatchNormalization

In [15]:
model = Sequential()
model.add(Conv2D(512, kernel_size=(3, 3), activation='relu', input_shape=(128,128,3)))
model.add(BatchNormalization())
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(BatchNormalization())
model.add(Conv2D(256, kernel_size=(3, 3), activation='relu'))
model.add(BatchNormalization())
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(BatchNormalization())
model.add(Flatten())
model.add(Dense(128, activation='relu'))
model.add(BatchNormalization())
model.add(Dense(22, activation='softmax'))

In [16]:
from keras.callbacks import EarlyStopping
from keras.callbacks import ModelCheckpoint

In [17]:
filepath='//content//drive//MyDrive//Jai_Bert//checkpoints'

In [18]:
callback =EarlyStopping(
    monitor="val_loss",
    min_delta=0,
    patience=20,
    verbose=1,
    mode="auto",
    baseline=None,
    restore_best_weights=False,
)
checkpoint = ModelCheckpoint(filepath,monitor='val_loss',mode='min',save_best_only=True,verbose=1)


In [ ]:
model.compile(tf.keras.optimizers.Adam(lr=2e-5), loss='sparse_categorical_crossentropy', metrics=['accuracy'])

In [20]:
dev_model = model.fit(x=train_ds,
                               validation_data=val_ds,
                               epochs=100,
                               batch_size = 32,
                               verbose=1,
                               callbacks=[callback,checkpoint])

KeyboardInterrupt: ignored

In [ ]:
#Validation
loss, accuracy = model.evaluate(val_ds)
print(f'Loss: {loss}')
print(f'Accuracy: {accuracy}')

In [ ]:
import matplotlib.pyplot as plt

plt.plot(dev_model.history['accuracy'])
plt.plot(dev_model.history['val_accuracy'])
plt.ylabel('accuracy')
plt.xlabel('epoch')
plt.legend(['train','test'], loc='upper left')
plt.title('Training and Validation Accuracy')
plt.show()

In [ ]:
#Validation
loss, accuracy = model.evaluate(train_ds)
print(f'Loss: {loss}')
print(f'Accuracy: {accuracy}')

In [ ]:
plt.plot(dev_model.history['loss'])
plt.plot(dev_model.history['val_loss'])
plt.ylabel('loss')
plt.xlabel('epoch')
plt.legend(['train','test'], loc='upper right')
plt.title('Training and Validation Loss')
plt.show()

In [ ]:
#Validation
loss, accuracy = model.evaluate(test_ds)
print(f'Loss: {loss}')
print(f'Accuracy: {accuracy}')

In [ ]:
# Classify new data
reloaded_model = '/content/drive/MyDrive/Jai_Bert/checkpoints'
from tensorflow import keras

In [ ]:
reconstructed_model = keras.models.load_model(reloaded_model,compile=False)
reconstructed_model.compile(optimizer='adam',loss='sparse_categorical_crossentropy', metrics=['acc'])
tf.config.experimental_run_functions_eagerly(True)

In [ ]:
loss, accuracy = reconstructed_model.evaluate(val_ds)  #first 10k of checkpoints.
print(f'Loss: {loss}')
print(f'Accuracy: {accuracy}')

In [ ]:
img_url = "/content/drive/MyDrive/data set/train/Nail Fungus and other Nail Disease/acute-paronychia-1.jpg"

img = tf.keras.utils.load_img(
    img_url, target_size=(128,128)
)
img_array = tf.keras.utils.img_to_array(img)
img_array = tf.expand_dims(img_array, 0) # Create a batch

predictions = model.predict(img_array)
score = tf.nn.softmax(predictions[0])

print(
    "This image most likely belongs to {} with a {:.2f} percent confidence."
    .format(class_names[np.argmax(score)], 100 * np.max(score))
)

In [ ]:
from tensorflow.keras.models import load_model
model.save("classification-3.h5")
b = load_model("classification-3.h5")

In [ ]:
b.summary()